# Some basic test examples to run with exo21cmFAST

In [ ]:
import py21cmfast as p21c
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Some useful functions for the plots

def myfigure(xlims:list = None, ylims:list = None, logx:bool = True, logy:bool = True):
    
    fig = plt.figure(figsize=(5,4))
    ax = fig.gca()
    ax.grid(True, alpha = 0.5, linewidth=0.5)
    ax.grid(True, alpha = 0.5, linewidth=0.5)

    if logx is True:
        ax.set_xscale('log')
    if logy is True:
        ax.set_yscale('log')
        
    if xlims is not None:
        ax.set_xlim(xlims)
    if ylims is not None:
        ax.set_ylim(ylims)
    
    return fig, ax 


## Example 0: The case without exotic energy injection

In [ ]:
lightcone_no_exo = p21c.run_lightcone(
            redshift = 5,        # Minimal value of redshift -> Here we will go slightly lower (cannot go below z = 4 for DarkHistory)
            user_params = {
                "BOX_LEN":                  50,    # Default value: 300  (Box length Mpc) 1000
                "DIM":                      None,  # Default value: None / gives DIM=3*HII_DIM (High resolution) None
                "HII_DIM":                  20,    # Default value: 200  (HII cell resolution) 350
            },
            flag_options = {
                "USE_TS_FLUCT"               : True,  # Turn on IGM spin temperature fluctuations    
                "FORCE_DEFAULT_INIT_COND"    : True,  # Forces the code to use the default initial conditions from RECFAST 
            },
            lightcone_quantities = (),
            global_quantities    = ('brightness_temp', 'density', 'xH_box', 'x_e_box', 'Ts_box', 'Tk_box'),
            verbose_ntbk = True,
            direc='./cache', 
        )

In [ ]:
## Plotting the results

redshift_no_exo_arr   = lightcone_no_exo.node_redshifts
Tb_global_no_exo_arr  = lightcone_no_exo.global_quantities['brightness_temp']
xH_global_no_exo_arr  = lightcone_no_exo.global_quantities['xH_box']
xe_global_no_exo_arr  = lightcone_no_exo.global_quantities['x_e_box']
TS_global_no_exo_arr  = lightcone_no_exo.global_quantities['Ts_box']
TK_global_no_exo_arr  = lightcone_no_exo.global_quantities['Tk_box']

fig, ax = myfigure(xlims = [5, 35], ylims = [-180, 50], logx=False, logy=False)
ax.set_xlabel(r"$z$")
ax.set_ylabel(r"$\overline{T_{\rm b}} ~ \rm [mK]$")
ax.plot(redshift_no_exo_arr, Tb_global_no_exo_arr, 'k-', linewidth=1)
fig.savefig('figures/global_Tb_with_no_exo.pdf', bbox_inches='tight')

## Example 1: Using DarkHistory for $\chi \to e^+e^-$

In this example we evaluate the impact of dark matter decaying into electrons and positions. We choose a mass of $130$ MeV, a lifetime of $10^{26}$ s and we include the effect of backreaction. To save storage space we only output global quantities of the lightcone. To run this section you need to have installed DarkHistory and added its path to the $PYTHONPATH

In [ ]:
lightcone_DH, output_exotic_energy_injection_DH = p21c.run_lightcone(
            redshift = 5,        # Minimal value of redshift -> Here we will go slightly lower (cannot go below z = 4 for DarkHistory)
            user_params = {
                "BOX_LEN":                  50,    # Default value: 300  (Box length Mpc) 1000
                "DIM":                      None,  # Default value: None / gives DIM=3*HII_DIM (High resolution) None
                "HII_DIM":                  20,    # Default value: 200  (HII cell resolution) 350
            },
            astro_params = {
                "DM_LOG10_MASS"     : np.log10(1.3e+8),  # DM mass in eV
                "DM_LOG10_LIFETIME" : np.log10(1e+26),   # Lifetime | relevant only if DM_PROCESS = 'decay'
            },
            flag_options = {
                "USE_TS_FLUCT"               : True,          # Turn on IGM spin temperature fluctuations
                "USE_DM_ENERGY_INJECTION"    : True,          # Turn on DM energy injection
                "DM_PROCESS"                 : 'decay',       # Energy injection process 'swave', 'decay', ... 
                "DM_PRIMARY"                 : 'elec_delta',  # Primary particles (see list in user_params description)
                "DM_BACKREACTION"            : True,          # Turns on backreaction       
            },
            lightcone_quantities = (),
            global_quantities    = ('brightness_temp', 'density', 'xH_box', 'x_e_box', 'Ts_box', 'Tk_box'),
            verbose_ntbk = True,
            output_exotic_data=True,
            direc='./cache', 
        )

In [ ]:
## Plotting the results

redshift_DH_arr   = lightcone_DH.node_redshifts
Tb_global_DH_arr  = lightcone_DH.global_quantities['brightness_temp']
xH_global_DH_arr  = lightcone_DH.global_quantities['xH_box']
xe_global_DH_arr  = lightcone_DH.global_quantities['x_e_box']
TS_global_DH_arr  = lightcone_DH.global_quantities['Ts_box']
TK_global_DH_arr  = lightcone_DH.global_quantities['Tk_box']

fig, ax = myfigure(xlims = [5, 35], ylims = [-180, 50], logx=False, logy=False)
ax.set_xlabel(r"$z$")
ax.set_ylabel(r"$\overline{T_{\rm b}} ~ \rm [mK]$")
ax.plot(redshift_DH_arr, Tb_global_DH_arr, 'b-', linewidth=1)
ax.plot(redshift_no_exo_arr, Tb_global_no_exo_arr, 'k-', linewidth=1)
fig.savefig('figures/global_Tb_with_DH.pdf', bbox_inches='tight')


## Example 2: Using $f_{\rm heat}$ templates

In this example we evaluate the impact of dark matter decay with $f_{\rm heat}$ represented by a Schechter function, 
\begin{equation}
f_{\rm heat}(z) = f_0 e^{a (z-z_0)} \left( \frac{z}{z_0}\right)^b
\end{equation}
where $z_0=15$ and the parameters $(f_0, a, b)$ are set by hand (here matching values obtained from DarkHistory). We choose a mass of $10$ GeV (although irrelevant in that case for decay) and a lifetime of $10^{26}$ s. Here we also impose by hand initial conditions for $x_e$ and $T_{\rm K}$ (here matching values obtained from DarkHistory). The values of the deposition fractions in other canals are evaluated from $f_{\rm heat}$ according to
\begin{equation}
\begin{cases}
& f_{\rm ion, H} =  2.2 f_{\rm heat}\\
& f_{\rm ion, He} =  0.07 f_{\rm heat}\\
& f_{\rm exc} =  1.7 f_{\rm heat} \, .
\end{cases}
\end{equation}
Custom values for the coefficients of proportionality can be set in astro_params with the flag_option "USE_DM_CUSTOM_F_RATIOS"


In [ ]:
lightcone_templates, output_exotic_energy_injection_templates = p21c.run_lightcone(
            redshift = 5,        # Minimal value of redshift -> Here we will go slightly lower (cannot go below z = 4 for DarkHistory)
            user_params = {
                "BOX_LEN":                  50,    # Default value: 300  (Box length Mpc) 1000
                "DIM":                      None,  # Default value: None / gives DIM=3*HII_DIM (High resolution) None
                "HII_DIM":                  20,    # Default value: 200  (HII cell resolution) 350
            },
            astro_params = {
                "DM_LOG10_MASS"     : np.log10(1.3e+8),  # DM mass in eV
                "DM_LOG10_LIFETIME" : np.log10(1e+26),   # Lifetime | relevant only if DM_PROCESS = 'decay'

                "DM_FHEAT_APPROX_PARAM_LOG10_F0" : np.log10(0.1531),     # Parameter to feed to the template of fheat
                "DM_FHEAT_APPROX_PARAM_A"        : 0.003209,            # Parameter to feed to the template of fheat
                "DM_FHEAT_APPROX_PARAM_B"        : 0.1295,               # Parameter to feed to the template of fheat

                "LOG10_XION_at_Z_HEAT_MAX"      : np.log10(0.0031),      # Ionization fraction at redshift z = Z_HEAT_MAX = 35 (or max_redshift)
                "LOG10_TK_at_Z_HEAT_MAX"        : np.log10(118)          # Temperature in K at redshift z = Z_HEAT_MAX = 35 (or max_redshift)
            },
            flag_options = {
                "USE_TS_FLUCT"               : True,          # Turn on IGM spin temperature fluctuations
                "USE_DM_ENERGY_INJECTION"    : True,          # Turn on DM energy injection
                "USE_DM_EFFECTIVE_DEP_FUNCS" : True ,         # Treat the energy injection with approximate templates (instead of DarkHistory)
                "DM_PROCESS"                 : 'decay',       # Energy injection process 'swave', 'decay', ... 
                "DM_PRIMARY"                 : 'elec_delta',  # Primary particles (see list in user_params description)
                "DM_BACKREACTION"            : True,          # Turns on backreaction 
                "DM_FHEAT_APPROX_SHAPE"      : 'schechter',    # Shape of the template for f_heat
                "USE_CUSTOM_INIT_COND"       : True           # Forces the code to use the init conditions defined in astro_params
            },
            lightcone_quantities = (),
            global_quantities    = ('brightness_temp', 'density', 'xH_box', 'x_e_box', 'Ts_box', 'Tk_box'),
            verbose_ntbk = True,
            output_exotic_data=True,
            direc='./cache', 
        )

In [ ]:
## Plotting the results

redshift_templates_arr   = lightcone_templates.node_redshifts
Tb_global_templates_arr  = lightcone_templates.global_quantities['brightness_temp']
xH_global_templates_arr  = lightcone_templates.global_quantities['xH_box']
xe_global_templates_arr  = lightcone_templates.global_quantities['x_e_box']
TS_global_templates_arr  = lightcone_templates.global_quantities['Ts_box']
TK_global_templates_arr  = lightcone_templates.global_quantities['Tk_box']

## Plotting the brightness temperature
## Comment the corresponding lines if you have not run the first two examples before
fig, ax = myfigure(xlims = [5, 35], ylims = [-180, 50], logx=False, logy=False)
ax.set_xlabel(r"$z$")
ax.set_ylabel(r"$\overline{T_{\rm b}} ~ \rm [mK]$")
ax.plot(redshift_templates_arr, Tb_global_templates_arr, 'r-', linewidth=1)
ax.plot(redshift_DH_arr, Tb_global_DH_arr, 'b-', linewidth=1)
ax.plot(redshift_no_exo_arr, Tb_global_no_exo_arr, 'k-', linewidth=1)
fig.savefig('figures/global_Tb_with_templates.pdf', bbox_inches='tight')

## Plotting the spin and the IGM temperature
## Comment the corresponding lines if you have not run the first two examples before
fig, ax = myfigure(xlims = [5, 35], logx=False)
ax.set_xlabel(r"$z$")
ax.set_ylabel(r"$\overline{T} ~ \rm [K]$")
line1, = ax.plot(redshift_templates_arr, TS_global_templates_arr, 'r-', linewidth=1)
line2, = ax.plot(redshift_templates_arr, TK_global_templates_arr, 'r--', linewidth=1)
line1, = ax.plot(redshift_DH_arr, TS_global_DH_arr, 'b-', linewidth=1)
line2, = ax.plot(redshift_DH_arr, TK_global_DH_arr, 'b--', linewidth=1)
line1, = ax.plot(redshift_no_exo_arr, TS_global_no_exo_arr, 'k-', linewidth=1)
line2, = ax.plot(redshift_no_exo_arr, TK_global_no_exo_arr, 'k--', linewidth=1)
ax.legend([line1, line2], [r"$\rm T_{\rm S}$", r"$\rm T_{\rm K}$"])
fig.savefig('figures/global_TS_TK_with_templates.pdf', bbox_inches='tight')